In [2]:
import pandas as pd
import numpy as np

Load the movies dataset

In [3]:
movies = pd.read_csv('ml-latest/movies.csv')

In [4]:
ratings = pd.read_csv('ml-latest/ratings.csv')

In [5]:
tags = pd.read_csv('ml-latest/tags.csv')

In [6]:
links = pd.read_csv('ml-latest/links.csv')

In [7]:
genome_tags = pd.read_csv('ml-latest/genome-tags.csv')

In [8]:
genome_scores = pd.read_csv('ml-latest/genome-scores.csv')

Examine the datasets

In [9]:
movies.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [10]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


retrieve all the genres available in the movies table

In [11]:
genres = set(movies['genres'].str.cat(sep='|').split('|'))

In [15]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [16]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


Calculate average ratings per movie

In [18]:
movie_data = pd.merge(ratings, movies, on='movieId')
pd.DataFrame(movie_data.groupby('title')['rating'].mean().sort_values(ascending=False))

,rating
title,
Faintheart (2008),5.0
White Tiger (1996),5.0
Radio Unnameable (2012),5.0
Rafter Romance (1933),5.0
Who Killed Chea Vichea? (2010),5.0
"Gesuzza, La Sposa Garibaldina (1934)",5.0
Ram Dass: Fierce Grace (2001),5.0
Ramen Shop (2018),5.0
Ramona (1936),5.0


In [17]:
ratings_mean_count['rating_counts'] = pd.DataFrame(movie_data.groupby('title')['rating'].count())

NameError: name 'ratings_mean_count' is not defined

In [ ]:
tags.dtypes

In [ ]:
tags.head()

In [ ]:
links.dtypes

In [ ]:
links.head()

In [ ]:
genome_tags.dtypes

In [ ]:
genome_tags.head()

In [ ]:
genome_scores.dtypes

In [ ]:
genome_scores.head()